# 08. CODE SEARCH: APPROACHES

1. [x] введение
2. [x] запросы
3. [x] предобработка запросов
4. [x] индексирование, получение ответов
5. [x] ранжирование и фильтрация
6. [x] бенчмарки
7. [x] ссылки

На основе [Grazia Pradel - Code search A survey of techniques for finding code 2022](https://arxiv.org/abs/2204.02765)

# 1. Введение

Человечество накапливает огромное количество информации. Необходимо уметь выполнять поиск по этой информации. 

Программное обеспечение занимает важную роль в нашей жизни, программный код --- это отдельный вид информации, который требует специальных инструментов для поиска.

> Например, ядро Linux состоит из миллионов строк исходного кода.
> Другой пример, в год на GitHub-е появляется более 60 миллионов новых проектов.

Большое количество исходного кода приводит к ситуации, что большая часть кода, который пишут разработчики, уже ранее был написан. Или, как минимум, был написан код, похожий на данный.

Всё это является причиной того, что важно уметь эффективно искать по коду, чтобы:
- найти, где реализована нужная функциональность
- понять, что делает определённый фрагмент кода
- найти, где есть ещё подобный код, чтобы исправить ошибку
и так далее.

Для поиска по коду (code search) разрабатываются специальный инструменты --- поисковые движки (code search engines).

Что нужно хорошему поисковому движку?
- удобный интерфейс запросов
- поиск, который даёт релевантные ответы на запросы
- делает это эффективно.

Поиск по коду похож на поиск в корпусах на естественном языке, но есть отличия:
- код имеет определённый синтаксис, который однозначно интерпетируется
- синтаксис кода задаёт структуру (структурный свойства кода)
- код имеет вполне определённую семантику (функциональность) --- нет двусмыслености
- кроме того, часто код в одном репозитории написан на разных языках, используются разные соглашения по стилю.
- запросы часто формулируются не в том же языке программирования (естественный язык, специальный язык запросов, смесь языков)

Общая схема  поиска по коду

![overview](./res/08_code_search_overview.png)

# 2. Поисковые запросы

*Запрос (query)* --- явное выражение поискового намерения, используемое пользователем для поискового движка.

Что хотим от языка запросов?
1. Простота. Легко формулировать запрос
2. Выразительность. Язык запросов должен позволять определить то, что надо найти.
3. Точность. Запросы должны позволять выразить намерение настолько однозначно, насколько это возможно.

Будем различать следующие типы языков запросов:
1. Неформальные
  - в свободной форме
2. Формальные
  - использующие общеизвестные языки программирования
  - использующие специально разработанные языки программирования
  - использующие пары: вход -- выход
3. Гибридные

# 2.1 Языки запросов в свободной форме

> Например, `read file line by line`

Запрос описывает намерение на естественном языке. Могут быть элементы языка программирования.

> Например, `FileReader close`

Запросы в свободной форме чаще всего используются.

Преимущества:
- удобно пользователю

Недостатки
- естественные языки неоднозначный (например, *float*)
- словарь, используемый в запросах, может не соответствовать словарю кодовой базы (например, *array* и [.] в Python-е означают массив)

Проблему несоответствия словарей (например, поиск регулярных варажений) можно облегчить, если вложить слова естественного языка и слова языка программирования в общее векторное пространство (word2vec). Либо делать предобработку и расширение запросов (см. далее).

## 2.2. Запросы на основе общеизвестных языков программирования

Основные подходы:
- фрагменты кода
> Например, запрос --- это код с частичной реализацией, которую надо дописать
> ```
> try {
> File file = File . createTempFile (" foo " , " bar ") ;
> } catch ( IOException e ) { }
> ```
- код с плейсхолдерами --- явно указаны пропущенные фрагменты кода
> Пример.
>```
>public void actionClose ( JButton a , JFrame f ) {
>__CODE_SEARCH__ ;
>}
>```
- код с некоторыми паттернами --- бастрактный шаблоны кода для поиска
> Пример.
>```
>if (# = #) @ ;
>```

Недостатки:
- проблема парсить код существующими библиотеками (надо сначала сделать код синтаксически корректным)

Преимущества:
- можно использовать в рекомендательной системы во время написания кода (экономится время за запрос)

Есть подходы, когда в качестве запроса принимается бинарник.

## 2.3 Запросы на основе специально разработанных языков программирования

### Использование логических языков программирования

Предикаты, описывающие свойства кода.
> Например, найти пакет с классом, который называется `HelloWorld`:
>```
>package (? P , class , ? C ) , class (? C , name , HelloWorld )
>```

Иногда есть возможность задать более высокоуровневые свойства:
> Наример, 
>```
>import count > 5 AND extends class FooBar
>```

### Существенные расширения существующих языков


>Например, поиск вложенных `if-else`:
>```
>$ ( if $$ else $ ) $ + 
>```

### Другие специальные языки

Например, описание программы так, что описываются вычисления через граф вычислений. Можно искать независимо от языка.

## 2.4 Использование пары: вход -- выход

Подход интересен тем, что использует ключевую особенность кода --- исполнимость. Запрос через примеры описывает желаемое поведение.

> Например, для входа `abc@def.org` хотим на выходе иметь `abc`.

## 2.5. Гибридные запросы

Подходы комбинируют несколько вариантов, описанных выше.

> Например,
> ```
> sort playerScores in ascending order
> ```
> где `playerScores` --- переменная из кода.

# 3. Предообработка и расширение запросов

Проблема:
- запрос пользователя может не быть самым лучшим, чтобы получить то, что нужно.

Почему?
- неоднозначность естественного языка
- различие словарей запросов и кода
- пользователь может сам быть не уверен, что именно ему надо

Что делать? --- Улучшать запрос.
Выделяют следюущие компоненты:
- интерфейс пользователя
- какая дополнительня информация используется
- как модифицируется запрос

## 3.1 Интерфейс пользователя

Вовлечён ли пользователь в модифицирование запроса? Если "да", то как?

### Прозрачность для пользователя

Как правило пользователь не видит, как модифицируется запрос.

Но бывают варианты, когда пользователь интерактивно модифицирует запрос но основе предложенных рекомендаций.

### Обратная связь от пользователя

Есть разные варианты. Иногда обратная связь используется в явном виде: пользователь явно оценивает результаты поиска.
Иногда обратная связь используется неявно, когда запрос модифицируется на основе действий пользователя.

## 3.2 Используемые дополнительные источники информации

### Результаты первого поиска

Используются информация из первого запроса.

Недостаток:
- требуется несколько поисковых попыток, чтобы получить резульата

### Похожесть поисковых запросов

- Можно составить специальные словари терминов или синонимов, и заменять слова в запросе.
- Можно заменять похожие на основе эмбеддингов слова.

### NL/code данные

Использовать данные для исправления запроса
- API документация
- форумы

Можно находить дополнительные термины и добавлять в запрос.

## 3.3 Техники модифицирования запросов

- добавить веса для частей поискового запроса (на основе документации, например)
- добавить или убрать части запроса (например, на основе документации)
- заменить запрос на более эффективное представление (например, заменить на конечный автомат или граф)

# 4. Индексирование и поиск кода

Обычно всё делается в два шага:
- индексирование данных (числовой вектор)
- поиск релевантных данных

Задачи индексирования:
- должны уметь находить правильный код
- должны уметь делать это быстро


Общая схема:
![indexing](./res/08_code_search_indexing.png)

## 4.1 Что индексируем?

Варианты:
- Исходный код и бинарный код
- Runtime-поведение кода (функциональность)
- Информация на естественном языке, связанная с кодом (комментарии, API-документация, сообщения коммитов, обсуждения)

## 4.2 Представление информации

Есть три группы подходов:
- представление данных как отдельных элементов кода
- представление данных как последовательностей элементов кода
- представление данных как взаимодейсвтующих элементов кода

### Отдельные элементы кода

Игнорируем порядок, любые взаимосвязи. Фрагмент кода --- множество элементов кода.
Премущества:
- простота

Недостатки:
- теряется важная информация

### Последовательности элементов кода

Сохраняем информацию по порядке элементов (например, на основе AST).

### Взаимодействующие элементы кода

Сохраняется информация о классах, наследовании, вызовах, отношении "содержать".

## 4.3 Как сравниваем запросы и код?

- на основе векторов признаков
- на основе эмбеддингов
- на основе база данных
- на основе графов

###  На основе векторов признаков

Вычисляется расстояние (похожесть) между векторами.
Вычислять всевозможные попарные расстояния требует много времени.
Поэтому всё происходи в пространстве, где можно эффективно получить список похожих кандидатов на заданный вектор.

Используются:
- булевы вектора (есть узел AST-дерева или нет)
- на основе TF-IDF
и тд (R*-деревья, например).

### На основе обучения

Модель машинного обучения выучивается вкладывать запрос и код в одно векторное пространство.
Похожие близки, не похожие --- далеки. Есть датасеты для подобных задач.

### На основе базы данных

Запросы --- запросы к базе данных.

### На основе графов

Имеем представление запроса и кода с помощью графов. Тогда задача сводится к сравнению графов.

# 5. Ранжирование и фильтрация (pruning)

## 5.1 Ранжирование

- на основе расстояний метрического пространства
- специальный подходы (например, использовать расстояние для строк)
- с использованием машинного обучения (например, BERT файнтюнится для ранжирования)

## 5.2 Фильтрация (pruning)

- по порогу
- по количеству
- убрать похожие

# 6. Бенчмарки

- CodeSeachNet
- CodeXGLUE
- Search4Net
- CoSQA

# 7. Полезные ссылки

- [Grazia Pradel - Code search A survey of techniques for finding code 2022](https://arxiv.org/abs/2204.02765)
- [Васильев - Поиск и навигация в IDE](https://www.youtube.com/watch?v=rhMGmDfmMNo&list=PLjufDdskD5S6jwrQZP-ikUtNWnRj5xNuN&index=18)